#Deep Learning for Healthcare -- Team 44<br>
Replicated Paper: An attention based deep learning model of clinical events in the intensive care unit<br>
<br>
Github Repo: https://github.com/jgbyc/DL4H_Team_44.git
<br><br>
Kun Ren (kunren2@illinois.edu)<br>
Yichen Bi (ybi5@illinois.edu)

#Introduction
* Background of the problem
  * What type of problem?<br>
  Develop and implement predictive models and computer assisted diagnostic (CAD) solutions that are interpretable, generalizable, and able to predict important clinical outcomes to aid in clinical care and provide high level supportive analytics for intensive care units (ICUs)<br><br>
  * What is the importance/meaning of solving the problem?<br>
  It is important to prevent information overload to ensure safe and efficient delivery of patient care. Reducing information overload is associated with more rapid care with fewer errors.<br><br>
  * What is the difficulty of the problem?<br>
  Traditional deep learning models are difficult to implement due to their reliance on large datasets which are rare in healthcare. Moreover, these models often exhibit limited interpretability, which have driven the adoption of transparent linear models.<br><br>
  * The state-of-the-art methods and effectiveness.
  Two popular methods are available: <br><br>
  Scoring system. Scoring methods are usually trained on one outcome and their effectiveness will be significantly compromised when utilized for another for which they were not designed.<br>
  Traditional RNN models. Despite the strength of RNN models, their model interpretability are generally low. <br><br>
* Paper explanation
  * What did the paper propose?<br>
   The paper “An attention based deep learning model of clinical events in the intensive care unit” explores the application of advanced deep learning techniques to the critical field of healthcare. It introduces a model that employs Long Short-Term Memory (LSTM) networks enhanced with an attention mechanism to analyze clinical events within the ICU setting. The model is trained on the MIMIC-III dataset, aiming to predict the daily likelihood of sepsis, myocardial infarction, and the need for vancomycin antibiotic administration3.

  * What is the innovations of the method?<br>
  The paper uses a Long Short-Term Memory (LSTM) networks enhanced with an attention mechanism. This enhanced mechanism can significantly improve the model interpretability.

  * How well the proposed method work (in its own metrics)?<br>
  The AUC of same-day myocardial ischemia, sepsis and vancomycin administration are 0.834, 0.952 and 0.904 respectively.

  * What is the contribution to the research regime?<br><br>
  The contribution of this paper is twofold:<br>
  Innovative Use of Attention Mechanism: It leverages the attention mechanism to provide insights into which clinical data points are most influential in predicting outcomes, thereby offering a layer of interpretability often missing in complex models.<br>
  Clinical Decision Support: By accurately predicting critical clinical events, the model has the potential to serve as a decision support tool, helping to manage the vast amount of data ICU physicians must process, and ultimately improving patient care outcomes.<br>
  This paper stands out for its potential impact on the healthcare industry, providing a pathway towards more informed and data-driven clinical decisions in high-stakes ICU environments.


# Mount Notebook to Google Drive and Change the Workspace Folder
In order to run this notebook, preprocessed data in Pickled objects and pretrained modles must be stored under folder Colab_notebooks.

Below is the required file/folder structure.



Colab_notebooks<br>
├── pickled_objects<br>
├── saved_models<br>
├── figures<br>
├── keract.py<br>

This step will mount the notebook to Google Drive and change the Workspace to Colab_notebooks folder.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/MyDrive/Colab_Notebooks')

In [ ]:
import gc
import numpy as np
import pandas as pd
import keras.backend as K
from keras.models import load_model
from seaborn import heatmap
import seaborn as sns
import pickle
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis

#To visulize the attention layers and generate its heatmap, Keract module is imported from https://github.com/philipperemy/keract
import keract

# plot part.
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

# Scope of Reproducibility:
The original paper performs three experiments onmyocardial infarction, sepsis and vancomycin administration respectively with the same model strucuture. Due to the 8 min code running time limits, we will only reproduce and present the model results for vancomycin administration target.


1. Hypothesis 1: The LSTM-based model with an attention mechanism can accurately predict daily clinical event vancomycin administration in the ICU.
2. Hypothesis 2: The attention mechanism within the model provides interpretability by highlighting influential input variables that impact the model’s predictions.

3. Experiment 1: Replicate the model training using the MIMIC-III dataset to predict the daily occurrence of vancomycin administration. The performance will be measured using the area under the receiver operating characteristic curve (AUC).

4. Experiment 2: Generate attention maps to identify the variables that the model attends to most when making predictions. This will test the model’s interpretability and its potential utility for clinicians.

# Methodology

##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

### Source of the data
Our dataset was constructed by processing the MIMIC III Clinical Dataset. We have finished the training and been provided the access to the MIMIC III database.

In [ ]:
# Load the pre_processed MIMIC III data.
X_TRAIN_VANCOMYCIN = pickle.load(open('./pickled_objects/X_TRAIN_VANCOMYCIN.txt', 'rb'))
Y_TRAIN_VANCOMYCIN = pickle.load(open('./pickled_objects/Y_TRAIN_VANCOMYCIN.txt', 'rb'))

Y_VAL_VANCOMYCIN = pickle.load(open('./pickled_objects/Y_VAL_VANCOMYCIN.txt', 'rb'))
X_VAL_VANCOMYCIN = pickle.load(open('./pickled_objects/X_VAL_VANCOMYCIN.txt', 'rb'))

Y_TEST_VANCOMYCIN = pickle.load(open('./pickled_objects/Y_TEST_VANCOMYCIN.txt', 'rb'))
X_TEST_VANCOMYCIN = pickle.load(open('./pickled_objects/X_TEST_VANCOMYCIN.txt', 'rb'))

y_boolmat_test_VANCOMYCIN = pickle.load(open('./pickled_objects/y_boolmat_test_VANCOMYCIN.txt', 'rb'))
x_boolmat_test_VANCOMYCIN = pickle.load(open('./pickled_objects/x_boolmat_test_VANCOMYCIN.txt', 'rb'))

no_features_cols_VANCOMYCIN = pickle.load(open('./pickled_objects/no_feature_cols_VANCOMYCIN.txt', 'rb'))
features_VANCOMYCIN = pickle.load(open('./pickled_objects/features_VANCOMYCIN.txt', 'rb'))


my_cmap= sns.cubehelix_palette(14, as_cmap=True)
color_list = sns.cubehelix_palette(14)
color_list_reduced = sns.cubehelix_palette(7)
X_TRAIN = X_TRAIN_VANCOMYCIN
X_VAL = X_VAL_VANCOMYCIN
Y_TRAIN = Y_TRAIN_VANCOMYCIN
Y_VAL = Y_VAL_VANCOMYCIN
Y_TEST = Y_TEST_VANCOMYCIN
X_TEST = X_TEST_VANCOMYCIN
y_boolmat_test = y_boolmat_test_VANCOMYCIN
x_boolmat_test = x_boolmat_test_VANCOMYCIN
features = features_VANCOMYCIN

Y_TOTAL = np.concatenate([Y_TRAIN, Y_VAL, Y_TEST], axis=0)

### Statistics:


In [ ]:
# Show total number of VANCOMYCIN labels
Y_VANCOMYCIN = np.concatenate([Y_TRAIN_VANCOMYCIN, Y_VAL_VANCOMYCIN], axis=0)
print(np.sum(np.sum(Y_VANCOMYCIN.squeeze(), axis=1) > 0))

In [ ]:
# Show number of training dataset used for VANCOMYCIN administrtion
#print(no_features_cols_VANCOMYCIN)

print(X_TRAIN_VANCOMYCIN.shape)

### Data Process:
Individual patient ICU admissions 2 days or longer were identified. ICU stays longer than 14 days were truncated to the first 14 days. When a patient’s stay was fewer than 14 days, extra days were masked out with 0 in training.

The target variables are constructed when receipt of any dose of vancomycin.<br>
Predictor are similarly constructed and variables whose more than 75% values are missing are excluded. Totally, 119 features are selected for the modeling purpose. These 119 features are processed into 225 variables by using statistical conversions (average, max, min, std).<br><br>

The data have been preprocessed offline and the processed data are stored in pickle formats under pickled_objects folder

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

### Model Architecture
Input -> Attention Layers -> Masking Layers -> 256 Units LSTM Layers -> Time Distributed Layers -> Fully Connected Layer (Sigmoid Activation)

### Training Objectives
The model uses:<br>
* Optimizer that implements the RMSprop algorithm.
* Loss function that implements binary cross entropy.

Training Code rnn_mimic.py can be found in Github repo https://github.com/jgbyc/DL4H_Team_44.git

### Pre-trained models
Due to the 8 min notebook running limits, the model is trained outside the notebook and store in the folder saved_models.

In [ ]:
target = 'VANCOMYCIN'
TIME_STEPS = X_VAL.shape[1] #number of time_steps

In [ ]:
#function to load the pre-trained models
def return_loaded_model(model_name):
  loaded_model = load_model("./saved_models/{0}.h5".format(model_name))
  return loaded_model

In [ ]:
# The models are trained offline. Here we load the model from Google Drive
# Models trained with various fractions of training data
m = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14")
m_80 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_80_percent")
m_60 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_60_percent")
m_40 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_40_percent")
m_20 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_20_percent")
m_10 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_10_percent")
m_5 = return_loaded_model(model_name="kaji_mach_final_no_mask_VANCOMYCIN_pad14_5_percent")

# Evaluation and Results
The results includes two parts
* AUC Results to show the model accuracy.
* Heatmap to visulize the attention weights on each features on each day.


## AUC Results
Calculate the AUC results for models trained with various fractions of data


In [ ]:
# Finish the prediction by using the test data and pre-trained models
X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_mask_0 = m.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_80_percent = m_80.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_60_percent = m_60.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_40_percent = m_40.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_20_percent = m_20.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_10_percent = m_10.predict(X_TEST_MASK)
del X_TEST_MASK

X_TEST_MASK = np.copy(X_TEST)
X_TEST_MASK[x_boolmat_test] = 0
Y_PRED_5_percent = m_5.predict(X_TEST_MASK)
del X_TEST_MASK

In [ ]:
# Generate the ROC curve for models trained with various fractions of training data.
(fpr_100, tpr_100, thresholds_100) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_mask_0[~y_boolmat_test])
(fpr_80, tpr_80, thresholds_80) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_80_percent[~y_boolmat_test])
(fpr_60, tpr_60, thresholds_60) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_60_percent[~y_boolmat_test])
(fpr_40, tpr_40, thresholds_40) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_40_percent[~y_boolmat_test])
(fpr_20, tpr_20, thresholds_20) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_20_percent[~y_boolmat_test])
(fpr_10, tpr_10, thresholds_10) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_10_percent[~y_boolmat_test])
(fpr_5, tpr_5, thresholds_5) = roc_curve(Y_TEST[~y_boolmat_test], Y_PRED_5_percent[~y_boolmat_test])

fpr_tprs = [(fpr_100, tpr_100), (fpr_80, tpr_80), (fpr_60, tpr_60),
            (fpr_40, tpr_40), (fpr_20, tpr_20), (fpr_10, tpr_10), (fpr_5, tpr_5)]

fig, ax = plt.subplots()
ax.set_facecolor('white')

for color, fpr_tpr_tuple, i in zip(np.array(color_list_reduced)[::-1], fpr_tprs, [100,80,60,40,20,10,5]):
    plt.plot(fpr_tpr_tuple[0], fpr_tpr_tuple[1], label='{0}% of Training Data'.format(i), color=color)

plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.xlabel('False Positive Rate', fontsize=15)
plt.ylabel('True Positive Rate', fontsize=15)
plt.axhline(0, color='black')
plt.axvline(0, color='black')
legend = plt.legend(loc="lower right", prop={'size': 10}, bbox_to_anchor=(1.48, 0.05))
plt.savefig('./figures/{0}_less_data_roc_curves_Supplemental_Figure_2.svg'.format(target), format='svg',
             dpi=300, facecolor='white', transparent=True, bbox_extra_artists=(legend,), bbox_inches='tight')

In [ ]:
# Print AUC score for models with various fractions of training data.
print('The AUC score of 100% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_mask_0[~y_boolmat_test])))
print('The AUC score of 80% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_80_percent[~y_boolmat_test])))
print('The AUC score of 60% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_60_percent[~y_boolmat_test])))
print('The AUC score of 40% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_40_percent[~y_boolmat_test])))
print('The AUC score of 20% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_20_percent[~y_boolmat_test])))
print('The AUC score of 10% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_10_percent[~y_boolmat_test])))
print('The AUC score of  5% data is {0}'.format(roc_auc_score(Y_TEST[~y_boolmat_test], Y_PRED_5_percent[~y_boolmat_test])))
print('\n')

# Compute the metrics for model trained with 100% data
TN, FP, FN, TP = confusion_matrix(Y_TEST[~y_boolmat_test], np.around(Y_PRED_mask_0[~y_boolmat_test])).ravel()
PPV = TP/(TP+FP)
print("PPV for full model is {0}".format(PPV))
CR = classification_report(Y_TEST[~y_boolmat_test], np.around(Y_PRED_mask_0[~y_boolmat_test]))

print("Sensitivity for full model is {0}".format(CR.split('\n')[3].split()[2]))
# classification_report(Y_TEST[~y_boolmat_test], np.around(Y_PRED_mask_0[~y_boolmat_test]))

print(classification_report(Y_TEST[~y_boolmat_test], np.around(Y_PRED_mask_0[~y_boolmat_test])))

## Heatmap
Generate the heatmap to visulize the attention weights on each features on each day.

In [ ]:
# GET ACTIVATIONS MATRIX ; OUTPUT IS (BATCH_SIZE, TIME_STEPS, FEATURES)

activations = keract.get_activations(m, X_TEST, output_format='simple', layer_names='attention_vec')['attention_vec']
activations[x_boolmat_test] = np.nan

# AVERAGE THE ATTENTION MATRIX OVER FEATURES ; OUTPUT IS BATCH_SIZE, TIME_STEPS
attention_matrix = np.nanmean(activations, axis=2).squeeze()

# AVERAGE ATTENTION VECTOR ACROSS SAMPLES ; OUTPUT IS 1D TIME_STEPS
attention_vector_final = np.nanmean(attention_matrix, axis=0)

# This allows us to rank color intensity by activation. We sort the intensities, then argsort the indices ##
color_order = np.argsort(attention_vector_final, axis=0)
color_order_order = np.argsort(color_order)

In [ ]:
''' Due to the way features are selectd from the EMR and the fact potassium can be a
delivered medication or a lab value, special care was taken to ensure proper representation on heatmaps '''

if 'digoxin(?!.*fab)' in features:
    indexy = features.index('digoxin(?!.*fab)')
    features[indexy] = 'digoxin'

if 'potassium_y' in features:
    indexy = features.index('potassium_y')
    features[indexy] = 'potassium_med'

if 'potassium_x' in features:
    indexy = features.index('potassium_x')
    features[indexy] = 'potassium'

if 'cipfloxacin' in features:
    indexy = features.index('cipfloxacin')
    features[indexy] = 'ciprofloxacin'

features = [feature.lower() for feature in features]

In [ ]:
## FEATURES BY CATEGORY ##

cbc_diff_features = ['RBCs', 'WBCs', 'platelets', 'hemoglobin', 'hemocrit',
                              'atypical lymphocytes', 'bands', 'basophils', 'eosinophils', 'neutrophils',
                              'lymphocytes', 'monocytes', 'polymorphonuclear leukocytes']

vital_features = ['temperature (F)', 'heart rate', 'respiratory rate', 'systolic', 'diastolic',
                  'pulse oximetry']

lab_features = ['troponin', 'HDL', 'LDL', 'BUN', 'INR', 'PTT', 'PT', 'triglycerides', 'creatinine',
                  'glucose', 'sodium', 'potassium', 'chloride', 'bicarbonate',
                  'blood culture', 'urine culture', 'surface culture', 'sputum' +
                  ' culture', 'wound culture', 'Inspired O2 Fraction', 'central venous pressure',
                  'PEEP Set', 'tidal volume', 'anion gap']

demographic_features = ['age', 'm', 'black', 'daily weight', 'tobacco', 'diabetes', 'history of CV events']

med_features = ['epoetin', 'warfarin', 'heparin', 'enoxaparin', 'fondaparinux',
                                      'asprin', 'ketorolac', 'acetominophen',
                                      'insulin', 'glucagon',
                                      'potassium_med', 'calcium gluconate',
                                      'fentanyl', 'magensium sulfate',
                                      'D5W', 'dextrose',
                                      'ranitidine', 'ondansetron', 'pantoprazole', 'metoclopramide',
                                      'lisinopril', 'captopril', 'statin',
                                      'hydralazine', 'diltiazem',
                                      'carvedilol', 'metoprolol', 'labetalol', 'atenolol',
                                      'amiodarone', 'digoxin',
                                      'clopidogrel', 'nitroprusside', 'nitroglycerin',
                                      'vasopressin', 'hydrochlorothiazide', 'furosemide',
                                      'atropine', 'neostigmine',
                                      'levothyroxine',
                                      'oxycodone', 'hydromorphone', 'fentanyl citrate',
                                      'tacrolimus', 'prednisone',
                                      'phenylephrine', 'norepinephrine',
                                      'haloperidol', 'phenytoin', 'trazodone', 'levetiracetam',
                                      'diazepam', 'clonazepam',
                                      'propofol', 'zolpidem', 'midazolam',
                                      'albuterol', 'ipratropium',
                                      'diphenhydramine',
                                      '0.9% Sodium Chloride',
                                      'phytonadione',
                                      'metronidazole',
                                      'cefazolin', 'cefepime', 'vancomycin', 'levofloxacin',
                                      'ciprofloxacin', 'fluconazole',
                                      'meropenem', 'ceftriaxone', 'piperacillin',
                                      'ampicillin-sulbactam', 'nafcillin', 'oxacillin',
                                      'amoxicillin', 'penicillin', 'SMX-TMP']

cbc_diff_features = [[i.lower(), i.lower()+'_min', i.lower()+'_max', i.lower()+'_std'] for i in cbc_diff_features]
vital_features = [[i.lower(), i.lower()+'_min', i.lower()+'_max', i.lower()+'_std'] for i in vital_features]
lab_features = [[i.lower(), i.lower()+'_min', i.lower()+'_max', i.lower()+'_std'] for i in lab_features]
demographic_features = [i.lower() for i in demographic_features]
med_features = [i.lower() for i in med_features]

cbc_diff_feature_array = np.array(cbc_diff_features).flatten()
vital_features_array = np.array(vital_features).flatten()
lab_features_array = np.array(lab_features).flatten()
demographic_feature_array = np.array(demographic_features).flatten()
med_features_array = np.array(med_features).flatten()

features_built = np.hstack([cbc_diff_feature_array,vital_features_array,
                            lab_features_array,demographic_feature_array,med_features_array])

features_built_reduced = [i for i in features_built if i in features]

In [ ]:
## Identifies the index in the features list in the desired order ##
arranged_indices = [features.index(i) for i in features_built_reduced]

In [ ]:
## GET HEATMAPS FOR ALL INCIDENTS AVERAGE ##

plt.figure(figsize = (8,20))

sns.set(font_scale = 0.5)

heatmap(np.nanmean(activations,axis=0).T[arranged_indices],
        square=False,
        yticklabels=np.array(features)[arranged_indices],
        cmap=my_cmap)
plt.gcf().axes[-1].tick_params(labelsize=10)
plt.xticks(np.arange(TIME_STEPS)+0.5, np.arange(TIME_STEPS), fontsize=15) #, rotation=45,)
plt.xlabel('Day', fontsize=15)
plt.ylabel('Features', fontsize=20)
plt.savefig('./figures/heatmap_{0}_med_ranked_average_activation_Figure_3.eps'.format(target),
            format='eps', dpi=300, bbox_inches='tight')

**Note**: Heatmap is attached at the end of the report pdf file due to the printing issue of Colab. See appendix A

## Model comparison

### Comparing metrics with results of original paper
In the original paper, the AUC score of VANCOMYCIN administration when trained with 100% data is 0.904.<br>
Our experiment shows AUC score as 0.917 which slightly better than the orignal paper.
<br><br>
We also visually compare the heatmap with the orignal heatmap provided by the paper. It is very close.

# Discussion

So far the paper is reproducible. However our project has not been finished.
<br>
In the final reports, we will perform below work.
* Alter the hyperparameters to see its effects on the model
* Perform the ablation study by removing attention layer, reduce the LSTM units numbers.




# References

1. Kaji, D., Zech, J. R., Kim, J., Cho, S. K., Dangayach, N., Costa, A., & Oermann, E. K. (2019). An attention based deep learning model of clinical events in the intensive care unit. PLOS ONE, 14(2), e0211057. https://doi.org/10.1371/journal.pone.0211057
2. Keract: Keras Activations + Gradients (https://github.com/philipperemy/keract)



# Appendix A

Heatmap of all features

